In [2]:
#Importe des fonctions utiles
import requests
import pandas as pd
import io
import collections
from itertools import combinations
import networkx as nx
pd.options.display.max_colwidth = 100
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
#Importe les fonctions pour travailler avec l'API de HAL, pour les détails, voir
# https://colab.research.google.com/github/Yquetzal/Teaching_notebooks/blob/main/Networks/cours_HAL_network.ipynb#scrollTo=d1ee2598

!wget https://raw.githubusercontent.com/Yquetzal/Teaching_notebooks/main/Networks/HALfunctions.py


--2023-11-27 12:40:16--  https://raw.githubusercontent.com/Yquetzal/Teaching_notebooks/main/Networks/HALfunctions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9183 (9.0K) [text/plain]
Saving to: ‘HALfunctions.py.1’

HALfunctions.py.1   100%[===================>]   8.97K  --.-KB/s    in 0s      

2023-11-27 12:40:16 (44.0 MB/s) - ‘HALfunctions.py.1’ saved [9183/9183]



In [4]:
from HALfunctions import get_all_elements, co_occurence_network,co_occurence_network_laboratory,column_most_common,bi_partite_network,bi_partite_network_generic

# Faire la requête

In [8]:
# Here, detail the request. This exmaple: articles whose abstract contains "easy", 
# published between 2020 et 2022, at l'université de Lyon.

words="(network)"
dates="[2021 TO 2022]"
columns="title_s,structAddress_s"

request = "http://api.archives-ouvertes.fr/search/?q="+words+"&fq=publicationDate_s:"+dates+"&fl="+columns
simple_example = get_all_elements(request)
simple_example.head(3)

Nombre d'éléments au total :  9030
Éléments récupérés :  1000
Éléments récupérés :  2000
Éléments récupérés :  3000
Éléments récupérés :  4000
Éléments récupérés :  5000
Éléments récupérés :  6000
Éléments récupérés :  7000
Éléments récupérés :  8000
Éléments récupérés :  9000
Éléments récupérés :  9030


,title_s,structAddress_s
0,A scalable sequence encoding for collaborative editing,Université de Nantes – faculté des Sciences et Techniques (FST)2 Chemin de la HoussinièreBP 9220...
1,Bilevel Network Design,"Parc Scientifique de la Haute Borne 40\, avenue Halley Bât.A\, Park Plaza 59650 Villeneuve d'Asc..."
2,Efficient description of experimental effects in amplitude analyses,"163\, avenue de Luminy - Case 902 - 13288 Marseille cedex 09,Aix-Marseille UniversitéJardins du ..."


In [18]:
simple_example.iloc[1]['structAddress_s']

"Parc Scientifique de la Haute Borne 40\\, avenue Halley Bât.A\\, Park Plaza 59650 Villeneuve d'Ascq,Domaine de VoluceauRocquencourt - BP 10578153 Le Chesnay Cedex,Avenue Franklin Roosevelt 50 - 1050 Bruxelles,Université de Lille - Campus scientifique - Bâtiment ESPRIT - Avenue Henri Poincaré - 59655 Villeneuve d’Ascq,École Centrale de Lille - Cité Scientifique - CS 20048 59651 Villeneuve d'Ascq Cedex,EPE Université de Lille. -- 42 rue Paul Duez\\, 59000 Lille,Pavillon André-Aisenstadt\\, bureau 3520 2920\\, Chemin de la Tour Montréal (Québec) H3T 1J4 CANADA,Campus de l'Université de Montréal - 2500\\, chemin de Polytechnique - Montréal (Québec) H3T 1J4,2900 Boulevard Edouard-Montpetit\\, Montréal\\, QC H3T 1J4,3000 Chemin de la Côte-Sainte-Catherine\\, Montréal\\, QC H3T 2A7"

In [64]:
#extract the list of zip codes in column "structAddress_s"
zipcodes = simple_example["structAddress_s"].str.extractall(r"(\d{5})")

#count occurences of zip codes
zipcodes_count = zipcodes[0].value_counts()

#put the result in a dataframe
zipcodes_count = pd.DataFrame(zipcodes_count)


In [65]:
zipcodes_count_top = zipcodes_count[:50]


In [76]:
def get_coord_from_one_zipcode(zipcode):
    """
    Get the coordinates of a zip code, using the API https://geo.api.gouv.fr/adresse
    """
    url = "https://api-adresse.data.gouv.fr/search/?q="+zipcode+"&postcode="+zipcode
    r = requests.get(url)
    if r.status_code == 200:
        try :
            
            r.json()["features"][0]['geometry']['coordinates']
            return r.json()["features"][0]['geometry']['coordinates']

        
        except:
            return None
    else:
        return None
    
def get_all_coords(zipcodes_df):
    """
    Get the coordinates of all zip codes in a dataframe
    """
    coord = []
    for zipcode in zipcodes_df.index:
        coord.append(get_coord_from_one_zipcode(zipcode))
    return coord

In [77]:
zipcodes_count_top["coord"] = get_all_coords(zipcodes_count_top)

#inverse the coordinates, because the API gives longitude first, then latitude
zipcodes_count_top["coord"] = zipcodes_count_top["coord"].apply(lambda x: x[::-1])

In [78]:
zipcodes_count_top

,count,coord
0,,
75006,3453,"[48.849268, 2.330883]"
75013,2873,"[48.830193, 2.365001]"
31000,1983,"[43.606374, 1.42198]"
75014,1725,"[48.830108, 2.323026]"
91190,1401,"[48.695026, 2.130573]"
30122,1334,"[44.112566, 3.722123]"
40700,1139,"[43.652177, -0.593313]"
54500,962,"[48.661501, 6.162621]"
75005,808,"[48.845277, 2.350554]"


In [80]:
#remove the zip codes for which we did not find coordinates
zipcodes_count_top = zipcodes_count_top.dropna()

#plot  on a map of france
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

m = folium.Map(location=[46.5, 2], zoom_start=6)
#plot one marker for each zip code, with size proportional to the number of occurences
for index, row in zipcodes_count_top.iterrows():
    folium.CircleMarker(location=row['coord'], radius=row[0]/100, color="#3186cc", fill=True).add_to(m)
m
